In [1]:
import numpy as np 
import pandas as pd
import os
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from scipy import stats
import seaborn as sns; sns.set()
import warnings
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.utils import shuffle
warnings.filterwarnings(action = 'ignore') 
import gensim 
from gensim.models import Word2Vec
import sys
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import BaggingClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data=pd.read_csv('/content/drive/MyDrive/Exosomes/train_top10_DPC.csv')
data2=pd.read_csv('/content/drive/MyDrive/Exosomes/train_top100.csv')
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [5]:
print(X)

      AAC_I  SER_S  SER_C  QSO1_SC_H  ...  DPC1_YT  DPC1_YV  DPC1_YW  DPC1_YY
0      4.56 -0.283 -0.141     0.8630  ...     0.19     0.28      0.0     0.00
1      1.13 -0.283 -0.024     0.1637  ...     0.00     0.00      0.0     0.00
2      2.55 -0.271 -0.024     0.4981  ...     0.00     0.00      0.0     0.28
3      3.47 -0.283 -0.364     1.5386  ...     0.00     0.00      0.0     0.00
4      4.74 -0.265 -0.157     0.6882  ...     0.26     0.53      0.0     0.00
...     ...    ...    ...        ...  ...      ...      ...      ...      ...
5948   3.33 -0.310 -0.052     0.5378  ...     0.00     0.00      0.0     0.00
5949   2.08 -0.427 -0.053     0.4248  ...     0.30     0.00      0.0     0.00
5950   3.36 -0.239 -0.165     0.4542  ...     0.00     0.31      0.0     0.00
5951   4.88 -0.296 -0.089     0.9552  ...     0.29     0.20      0.1     0.00
5952   3.16 -0.268 -0.089     0.7149  ...     0.00     0.23      0.0     0.00

[5953 rows x 410 columns]


In [ ]:
#Forest = RandomForestClassifier(random_state = 42)
#n_estimators = [200, 400, 600, 800, 1000, 1200]
#max_depth = [5, 8, 15, 25, 30]
#min_samples_split = [2, 5, 10, 15, 100]
#min_samples_leaf = [1, 2, 5, 10] 

#hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
#              min_samples_split = min_samples_split, 
#             min_samples_leaf = min_samples_leaf)

#gridF = GridSearchCV(Forest, hyperF, cv = 3, verbose = 1, 
#                      n_jobs = -1)
#bestF = gridF.fit(X_train, y_train)

from sklearn.model_selection import StratifiedKFold

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(clf, param_distributions=params, n_iter=param_comb,
                                   scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3,
                                   random_state=1001 )
# Here we go
#start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


ValueError: ignored

In [ ]:
#FOR AUC
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10, random_state=42, shuffle=True)
kf.get_n_splits(X)
cc = []
dd = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf = RandomForestClassifier(random_state = 42, n_estimators = 100)
    clf.fit(X_train,y_train)
    cc.append(clf.predict_proba(X_test))
    dd.append(y_test)
    flat_pred = [x for sublist in cc for x in sublist]
    flat_true = [x for sublist in dd for x in sublist]

In [6]:

#FOR ACCURACY ETC
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10, random_state=42, shuffle=True)
kf.get_n_splits(X)
cc = []
dd = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Training
    clf = BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=False, n_estimators=450, random_state=6), n_estimators=25, n_jobs=-1, random_state=6, verbose = 3)
    clf.fit(X_train, y_train)
    cc.append(clf.predict(X_test))
    dd.append(y_test)
    flat_pred = [x for sublist in cc for x in sublist]
    flat_true = [x for sublist in dd for x in sublist]
print('Confusion Matrix:')
conf_matx= confusion_matrix(flat_true,flat_pred)
print('Recall:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[1][0]))
print('Sensitivity:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[0][1]))
print('Specificity:', conf_matx[1][1]/(conf_matx[1][1]+conf_matx[0][1]))
print('Accuracy:', (conf_matx[0][0]+conf_matx[1][1])/(conf_matx[1][1]+conf_matx[0][1]+conf_matx[1][0]+conf_matx[0][0]))

data3=pd.read_csv('/content/drive/MyDrive/Exosomes/test_top10_DPC.csv')
#data4=pd.read_csv('/content/drive/MyDrive/Exosomes/test_top100.csv')
z_pr = data3.iloc[:,:-1]
z_test = data3.iloc[:,-1]
z_pred=clf.predict(z_pr)
# print("Accuracy:",metrics.accuracy_score(z_test, z_pred))
aa = []
for i in range(0,len(z_test)):
  if z_test[i] == 'Y':
    aa.append(1)
  else:
    aa.append(0)

conf_matx= confusion_matrix(z_test,z_pred)
print('Recall:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[1][0]))
print('Sensitivity:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[0][1]))
print('Specificity:', conf_matx[1][1]/(conf_matx[1][1]+conf_matx[0][1]))
print('Accuracy:', (conf_matx[0][0]+conf_matx[1][1])/(conf_matx[1][1]+conf_matx[0][1]+conf_matx[1][0]+conf_matx[0][0]))

z_pred_1=clf.predict_proba(z_pr)
df123 = pd.DataFrame(z_pred_1)

#Training#
flat_pred_1 = pd.DataFrame(flat_pred)
from sklearn.metrics import roc_auc_score
train_auc = roc_auc_score(flat_true,flat_pred_1.iloc[:,-1].ravel())
print('train_auc: ', train_auc)

#Testing#
from sklearn.metrics import roc_auc_score
test_auc = roc_auc_score(z_test, clf.predict_proba(z_pr)[:, 1])
print('test_auc: ', test_auc)

from sklearn.metrics import matthews_corrcoef
mcc_train = matthews_corrcoef(flat_true, flat_pred_1)
mcc_test = matthews_corrcoef(z_test, z_pred)

print('mcc_train: ', mcc_train)
print('mcc_test: ', mcc_test)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.3s remaining:    0.0s


KeyboardInterrupt: ignored

In [ ]:
print (X_t)

      AAC_I  AAC_W  DPC1_AK  DPC1_AQ  ...  QSO1_G_I  QSO1_G_R  QSO1_SC1  QSO1_G1
0      4.56   0.93     0.09     0.09  ...       0.0    0.0001    0.9746   0.9746
1      1.13   0.56     0.28     0.28  ...       0.0    0.0001    0.9181   0.9181
2      2.55   0.28     0.00     0.00  ...       0.0    0.0001    0.9288   0.9288
3      3.47   0.69     0.70     0.00  ...       0.0    0.0000    0.8077   0.8077
4      4.74   0.53     0.53     0.26  ...       0.0    0.0001    0.9312   0.9312
...     ...    ...      ...      ...  ...       ...       ...       ...      ...
5948   3.33   0.93     0.00     0.37  ...       0.0    0.0001    0.9511   0.9511
5949   2.08   1.34     0.60     0.30  ...       0.0    0.0000    0.9528   0.9528
5950   3.36   0.31     0.61     0.31  ...       0.0    0.0001    0.9243   0.9243
5951   4.88   1.95     0.29     0.20  ...       0.0    0.0000    0.9742   0.9742
5952   3.16   0.90     0.68     0.23  ...       0.0    0.0001    0.9702   0.9702

[5357 rows x 1900 columns]


In [ ]:
def Model_1(train, test):
    ''' Trains the model and Saves the predictions in a CSV file
        train : Training set
        test : Test set
    '''
# Preprocessing
    X_train = OneHotEncoder(sparse = False).fit_transform([[x for x in s] for s in X])
    X_test = OneHotEncoder(sparse = False).fit_transform([[x for x in s] for s in y])
    Y_train = train['label']

    X_train, Y_train = RandomUnderSampler(random_state = 100).fit_resample(X_train, Y_train)
    X_train, Y_train = shuffle(X_train, Y_train, random_state = 100)

    # Training
    estimators = [
                ('rf', RandomForestClassifier(n_estimators = 300, max_depth = 45, min_samples_leaf = 7, random_state = 100)),
                ('mlp', MLPClassifier(max_iter = 200,random_state = 100)),
                ('knn', KNeighborsClassifier(n_neighbors = 4))]

    clf = StackingClassifier(estimators = estimators, final_estimator=LogisticRegression(random_state = 100),
                             n_jobs = -1, verbose = 1)
    clf.fit(X_train, Y_train)

    # Predicting
    Y_pred = clf.predict(X_test)
    Y_prob = [x[1] for x in clf.predict_proba(X_test)]
    result = pd.DataFrame()
    result["ID"] = test["ID"]
    result["Label"] = Y_prob
    result.to_csv("Submission_1.csv", index = False)
    result["Label"] = Y_pred
    result.to_csv("Predictions_1.csv", index = False)


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
data2=pd.read_csv('/content/test_DPC_AAC_PCP.csv')
z_pr = data2.iloc[:,:-1]
z_test = data2.iloc[:,-1]
z_pred=clf.predict(z_pr)
# print("Accuracy:",metrics.accuracy_score(z_test, z_pred))
aa = []
for i in range(0,len(z_test)):
  if z_test[i] == 'Y':
    aa.append(1)
  else:
    aa.append(0)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    8.9s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    8.9s finished


In [ ]:
print(np.shape(X))
print(np.shape(y))

(5953, 400)
(5953,)


In [ ]:
conf_matx= confusion_matrix(z_test,z_pred)
print('Recall:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[1][0]))
print('Sensitivity:', conf_matx[0][0]/(conf_matx[0][0]+conf_matx[0][1]))
print('Specificity:', conf_matx[1][1]/(conf_matx[1][1]+conf_matx[0][1]))
print('Accuracy:', (conf_matx[0][0]+conf_matx[1][1])/(conf_matx[1][1]+conf_matx[0][1]+conf_matx[1][0]+conf_matx[0][0]))

Recall: 0.83136
Sensitivity: 0.7545744989834446
Specificity: 0.6828078078078078
Accuracy: 0.7629987908101572


In [ ]:
z_pred_1=clf.predict_proba(z_pr)
df123 = pd.DataFrame(z_pred_1)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   20.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   20.4s finished


In [ ]:
#Testing#
from sklearn.metrics import roc_auc_score
roc_auc_score(z_test, clf.predict_proba(z_pr)[:, 1])

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   35.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   35.0s finished


0.8478225585401419

In [ ]:
#Training#
flat_pred_1 = pd.DataFrame(flat_pred)
from sklearn.metrics import roc_auc_score
roc_auc_score(flat_true,flat_pred_1.iloc[:,-1].ravel())

0.7033418057745342